#Utility

In [ ]:
#@title Unzippare file zip
#@markdown Fornendo un file zip, questo script ce lo estrae in un percorso specifico che noi vogliamo.

from google.colab import files
import zipfile
import os

# 1. Carica file zip
uploaded = files.upload()

# 2. Prende il nome del file caricato
zip_filename = next(iter(uploaded))

# 3. Specifica il percorso di destinazione per l'estrazione
destination_path = '/content/unzipped_files' #@param {type:"string"}

# 4. Crea la directory di destinazione se non esiste
os.makedirs(destination_path, exist_ok=True)

# 5. Estrae il file zip nel percorso specificato
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(destination_path)

print(f"✅ File estratti in: {destination_path}")


# Installazione librerie e configurazione

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

!pip install -U -qq git+https://github.com/huggingface/diffusers.git
!pip install -qq accelerate tensorboard transformers ftfy gradio
!pip install -qq "ipywidgets>=7,<8"
!pip install -qq bitsandbytes

Scelta di quale Training usare nell'allenamento se quello di Shivam Shrirao o il nostro, che varia per una diversa augmentation rispetto all'originale, ovvero avere immagini con: variazioni geometriche, variazioni luminose ed avere deformazioni prospettiche casuali.

In [4]:
import os

#@markdown:
scelta = "ShivamShrirao" #@param ["ShivamShrirao", "nostro"]

# Controllo ed eventuale eliminazione del file esistente
if os.path.exists("train_dreambooth.py"):
    os.remove("train_dreambooth.py")
    print("✅ File 'train_dreambooth.py' esistente rimosso.")

# Download del file in base alla scelta
if scelta == "ShivamShrirao":
    !wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
    print("📥 Scaricato il file da ShivamShrirao.")
elif scelta == "nostro":
    !wget -q https://github.com/luigifienga18/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
    print("📥 Scaricato il file dal repository nostro.")


✅ File 'train_dreambooth.py' esistente rimosso.
📥 Scaricato il file dal repository nostro.


In [ ]:
#@title Inserire il token per loggarsi su Hugginface 🤗

#@markdown Serve per scaricare correttamente i pesi di Stable Diffusion ed eventualmente salvarsi il modello dopo
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

In [ ]:
#@markdown Inserire il path di HF per scaricare i pesi del modello
MODEL_NAME = "sd-legacy/stable-diffusion-v1-5"#@param {type:"string"}

#@markdown Inserire il percorso della cartella di output

OUTPUT_DIR = "stable_diffusion_weights/zwx" #@param {type:"string"}

OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

# Training

Shivam Shirao mette a disposizione la seguente tabella per abilitare correttamente gli iperparametri necessari in base alla nostra configurazione.

| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | *FALSE*                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


Usare `--gradient_checkpointing` se si hanno a disposizione 9.92 GB VRAM.

Rimuovere `--use_8bit_adam` per una maggiore precisione ma richiede 15.79 GB VRAM con `--gradient_checkpointing` altrimenti 17.8 GB VRAM.

Rimuovere  `--train_text_encoder` per ridurre ulteriori consumi di memoria, al costo di poter degradare i risultati.

In [ ]:
#@title Lista dei multi concetti

concepts_list = [
    {
        "instance_prompt":      "photo of camidndr person",
        "class_prompt":         "photo of a person",
        "instance_data_dir":    "/content/data/cami",
        "class_data_dir":       "/content/data/person"
    },
    {
        "instance_prompt":      "photo of anrosci person",
        "class_prompt":         "photo of a person",
        "instance_data_dir":    "/content/data/andrea",
        "class_data_dir":       "/content/data/person"
    }
    ]

# Creazione delle cartelle
import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

In [ ]:
#@markdown Con l'avvio di questa cella puoi fornire per ogni soggetto le immagini associate di riferiemnto

#@markdown Se non si vuole eseguire questa cella bisogna manualmente inserire nei percorsi i file

import os
from google.colab import files
import shutil

for c in concepts_list:
    print(f"Uploading instance images for `{c['instance_prompt']}`")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)

In [ ]:
#@title Inizio dell'Allenamento

import torch

#Pulizia della memoria della GPU per liberare spazio inutilmente occupato
#da eventualmente training o inferenze precedenti
torch.cuda.empty_cache()


!python3 train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="main" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=1200 \
  --save_interval=10000 \
  --save_sample_prompt="photo of anrosci person" \
  --concepts_list="concepts_list.json"

# --max_train_steps deve essere sull'ordine dei 1200 per realizzare soggetti affidabili
# --learning_rate abbiamo notato che buoni allenamenti devono essere con valori 2e-6 o 1e-6
# --save_sample_prompt serve per generare degli esempi a modello allenato


In [ ]:
#@markdown Specifica il percorso di dove sono i pesi, altrimenti lascialo vuoto, prenderà automaticamente i pesi del modello che ci servono
WEIGHTS_DIR = "" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

#Inferenza

In [ ]:
#@markdown Griglia per visualizzare le immagine sintetizzate del modello.
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

weights_folder = OUTPUT_DIR
folders = sorted([f for f in os.listdir(weights_folder) if f != "0"], key=lambda x: int(x))

row = len(folders)
col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))
scale = 4
fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(weights_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')

plt.tight_layout()
plt.savefig('grid.png', dpi=72)

In [ ]:
#@markdown Se si sta utilizzando il seguente codice direttamente per l'inferenza bisogna specificare il percorso di dove sono memorizzati i pesi, altrimenti se si è appena completato un allenamento usare la variabile che già è inizializzata correttamente

#@markdown Per caricare dei pesi generici di un modello allenato in precedenza, specificare o il percorso di HF o il percorso locale di dove si trovano nella macchina.

#@markdown Scegli la sorgente del modello da caricare:
sorgente_modello = "Usa WEIGHTS_DIR" #@param ["Usa WEIGHTS_DIR", "Inserisci percorso manuale"]

if sorgente_modello == "Usa WEIGHTS_DIR":
    model_path = WEIGHTS_DIR
else:
    #@markdown Inserisci il percorso completo del modello da prelevare:
    model_path = "/content/mio_modello" #@param {type:"string"}

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

pipe = StableDiffusionPipeline.from_pretrained(
    model_path,
    safety_checker=None,
    torch_dtype=torch.float16
).to("cuda")

pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
g_cuda = None


In [ ]:
#@markdown Se si preferisce è possibile fornire un seed diverso da quello inizializzato
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [ ]:
#@title Esecuzione

prompt = "photo of luifienga person kissing anrosci person" #@param {type:"string"}
negative_prompt = "blurry, low quality, deformed, distorted, bad anatomy, extra limbs, out of frame" #@param {type:"string"}
num_samples = 4 #@param {type:"number"}
guidance_scale = 7 #@param {type:"number"}
num_inference_steps = 80 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)

# Upload su HUGGING FACE

In [ ]:
#@markdown Inserisci il tuo Hugging Face Token
hf_token = ""  #@param {type:"string"}

#@markdown Inserisci il percorso locale del modello addestrato
local_model_path = "/content/stable_diffusion_weights/zwx/1200"  #@param {type:"string"}

#@markdown Inserisci il nome del repository Hugging Face da creare o usare
repo_name = "Dreambooth-CamiAndrea-ShivamShrirao"  #@param {type:"string"}

#@markdown (Opzionale) Inserisci il nome dell'organizzazione se vuoi pubblicarlo lì
organization = ""  #@param {type:"string"}

from diffusers import DiffusionPipeline
from huggingface_hub import login, create_repo
import os

# Step 1: Login
login(token=hf_token)

# Step 2: Carica il pipeline addestrato
pipeline = DiffusionPipeline.from_pretrained(local_model_path)

# Step 3: Prepara il nome del repository
repo_id = f"{organization.strip() + '/' if organization.strip() else ''}{repo_name.strip()}"

# Step 4: Scrivi README.md
readme_path = os.path.join(local_model_path, "README.md")

#@markdown Per modificare la descrizione aprire la tendina di mostra codice e cambiarla
description = """
# Dreambooth-CamiAndrea-ShivamShrirao

Questo modello è stato addestrato usando DreamBooth su più concetti.

## Dettagli

- Basato su Stable Diffusion
- Realizza Foto di Camilla e Andrea
- Token Andrea: `anrosc person`
- Token Camilla: `camdand person`
- Addestrato con più soggetti per supportare la generazione multi-concept
- Usare i token personalizzati indicati nella documentazione
- Modello addestrato sul primo dataset di Dreambooth presente nel Drive

Per ogni dettaglio tecnico, controlla i file di configurazione.
"""

with open(readme_path, "w") as f:
    f.write(description)

# Step 5: Crea il repo (se non esiste già)
create_repo(repo_id, exist_ok=True)

# Step 6: Pubblica il modello su Hugging Face Hub
pipeline.push_to_hub(repo_id)

print(f"✅ Modello caricato su: https://huggingface.co/{repo_id}")
